In [0]:
!pip install pyodbc
import pyodbc
from pyspark.sql import *
import datetime
from dateutil.relativedelta import *

In [0]:
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")


In [0]:
%run /LIBS/class_root_mountpoint

In [0]:
%run /LIBS/class_datalake_ult_pasta

In [0]:
%run /LIBS/class_conexao

In [0]:
path_os = raiz_path().get_path()+'TRUSTED/DW/'

In [0]:
path_ml = raiz_path().get_path()+'TRUSTED/ML/PEDIDO_SUGERIDO/'

In [0]:
# Fato OV
df_OS = spark.read.format("delta").load(path_os+'FATO_OV')
df_OS.registerTempTable("FATO_OV")


# Dim Cliente
df_OS = spark.read.format("delta").load(path_os+'DIM_CLIENTE')
df_OS.registerTempTable("DIM_CLIENTE")

# Dim Produto
df_OS = spark.read.format("delta").load(path_os+'DIM_MATERIAL')
df_OS.registerTempTable("DIM_MATERIAL")

# Fato OV
df_OS = spark.read.format("delta").load(path_os+'FATO_OV')
df_OS.registerTempTable("Pedido_Sugerido")

In [0]:
ds_recomendacoes = spark.read.jdbc(url= conexao().sql_server_jdbc_url(), table = "dw.Recomen_Pedido_Sugerido", properties= conexao().sql_server_jdbc_connection_properties())
ds_recomendacoes.registerTempTable("ds_recomendacoes")

In [0]:
dat_carga = spark.sql(""" select max(DATA_CARGA) from FATO_OV""")

In [0]:
# mes_menos_6_ov = '2017-01-01'
# mes_menos_6_yyyyMMdd_ov = '20170101'

##CAPTURA O MES-6
mes_menos_6_ov = str(datetime.date(datetime.datetime.now().year,datetime.datetime.now().month,1) - relativedelta(months=6))
mes_menos_6_yyyyMMdd_ov = mes_menos_6_ov[:4]+mes_menos_6_ov[5:7]+mes_menos_6_ov[-6:]
Current_Date = str(datetime.date(datetime.datetime.now().year,datetime.datetime.now().month,datetime.datetime.now().day) - relativedelta(days= 5))

In [0]:
spark.sql("""
select 
 CLI.COD_CLIENTE
,CLI.COD_GRUPO_CLIENTE
,CLI.DES_GRUPO_CLIENTE_PT
,CLI.COD_ESCRITORIO_VENDAS
,CLI.COD_GRUPO_PRECO
,CLI.DES_GRUPO_PRECO_PT
,CLI.DES_ESCRITORIO_VENDAS_PT
,CLI.COD_UF
,CLI.DES_ORGANIZACAO_PT
--,MAT.COD_LINHA_ORCAMENTARIA
,MAT.COD_MATERIAL
--,MAT.COD_SUBLINHA_ORCAMENTARIA
--,MAT.DES_SUBLINHA_ORCAMENTARIA_PT
,MAT.DES_MATERIAL_PT
--,MAT.DES_PROCESSO_FABRIL_PT
,MAT.DES_PROCESSO_FABRICACAO_PT
,MAT.DES_SISTEMA_PT
--,MAT.DES_SUBSISTEMA_PT
,OV.COD_SETOR_ATIVIDADE
,OV.COD_ORGANIZACAO_VENDA
,SUM(OV.VAL_ORDEM_BRUTO_REAL) AS VAL_ORDEM_BRUTO


        
from FATO_OV OV
inner join DIM_CLIENTE CLI ON OV.DIM_CLIENTE_ID = CLI.DIM_CLIENTE_ID
AND to_date(OV.DAT_ORDEM,'yyyyMMdd') BETWEEN CLI.DATA_INICIO_VIGENCIA AND COALESCE(CLI.DATA_FIM_VIGENCIA,'9999-12-31')
inner join DIM_MATERIAL MAT ON OV.DIM_MATERIAL_ID = MAT.DIM_MATERIAL_ID
Where OV.DAT_ORDEM >= '{0}'
Group By 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
""".format(mes_menos_6_ov)).registerTempTable("TEMP_FATO_OV")

In [0]:
filtro_vaj = "COD_ORGANIZACAO_VENDA = 'TBR1' and (lower(DES_GRUPO_CLIENTE_PT) like '%varejo%' or lower(DES_GRUPO_CLIENTE_PT) like '%vrj%')"

spark.sql("""
with total_vendas as
(  
    select sum(VAL_ORDEM_BRUTO) as total
    from TEMP_FATO_OV
    Where {0}
    
)
select COD_MATERIAL, 
        'TBR1_Varejo' as Tipo,
        sum(vendas) over (order by vendas desc ) * 1.0 / total as perc_vendas,
        if(sum(vendas) over (order by vendas desc) * 1.0 / total <= 0.8,1,0) as  top_80,
        if(sum(vendas) over (order by vendas desc) * 1.0 / total <= 0.9,1,0) as  top_90
from (
    select COD_MATERIAL, 
        sum(VAL_ORDEM_BRUTO) as vendas,
        (select total from total_vendas) as total
    from TEMP_FATO_OV
    Where {0}
    group by COD_MATERIAL
    order by 2 desc
    ) vendas_material
    order by perc_vendas desc
""".format(filtro_vaj)).registerTempTable("TEMP_TOP_MAT_VAREJO")

In [0]:

DATASET_VARJ_TBR1_PS = spark.sql("""
select a.*,
 Tipo,
 perc_vendas,
 top_80,
 top_90
from TEMP_FATO_OV a
inner join TEMP_TOP_MAT_VAREJO b ON a.COD_MATERIAL = b.COD_MATERIAL
Where {0}
""".format(filtro_vaj))
DATASET_VARJ_TBR1_PS.registerTempTable("DATASET_VARJ_TBR1_PS")


In [0]:
try: #entra aqui caso o arquivo ja exista
  dbutils.fs.ls(path_ml+'DATASET_VARJ_TBR1_PS')
  
  #replace caso ja exista dado em alguma data
  DATASET_VARJ_TBR1_PS.withColumn("VAL_ORDEM_BRUTO",DATASET_VARJ_TBR1_PS.VAL_ORDEM_BRUTO.cast('double'))\
    .withColumn("perc_vendas",DATASET_VARJ_TBR1_PS.perc_vendas.cast('double'))\
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("delta.autoOptimize.optimizeWrite","true")\
    .option("delta.autoOptimize.autoCompact" , "true")\
    .save(path_ml+'DATASET_VARJ_TBR1_PS')

except Exception as e: #entra aqui caso seja a primeira execucao - arquivo nao existe e ira ser criado
  if 'java.io.FileNotFoundException' in str(e):
    DATASET_VARJ_TBR1_PS.withColumn("VAL_ORDEM_BRUTO",DATASET_VARJ_TBR1_PS.VAL_ORDEM_BRUTO.cast('double'))\
    .withColumn("perc_vendas",DATASET_VARJ_TBR1_PS.perc_vendas.cast('double'))\
    .write \
    .format("delta") \
    .option("delta.autoOptimize.optimizeWrite","true")\
    .option("delta.autoOptimize.autoCompact" , "true")\
    .save(path_ml+'DATASET_VARJ_TBR1_PS')

In [0]:
filtro_fpp = "COD_ORGANIZACAO_VENDA = 'PBR1' and (lower(DES_ESCRITORIO_VENDAS_PT) like '%artistico%' or lower(DES_ESCRITORIO_VENDAS_PT) like '%tintas%' or lower(DES_ESCRITORIO_VENDAS_PT) like '%varejo%' )"

spark.sql("""
with total_vendas as
(  
    select sum(VAL_ORDEM_BRUTO) as total
    from TEMP_FATO_OV
    Where {0}
    
)
select COD_MATERIAL, 
        'PBR1_Tintas' as Tipo,
        sum(vendas) over (order by vendas desc ) * 1.0 / total as perc_vendas,
        if(sum(vendas) over (order by vendas desc) * 1.0 / total <= 0.8,1,0) as  top_80,
        if(sum(vendas) over (order by vendas desc) * 1.0 / total <= 0.9,1,0) as  top_90
from (
    select COD_MATERIAL, 
        sum(VAL_ORDEM_BRUTO) as vendas,
        (select total from total_vendas) as total
    from TEMP_FATO_OV
    Where {0}
    group by COD_MATERIAL
    order by 2 desc
    ) vendas_material
    order by perc_vendas desc
""".format(filtro_fpp)).registerTempTable("TEMP_TOP_MAT_FPP")

In [0]:
DATASET_FPP_PBR1_PS = spark.sql("""
select a.*,
 Tipo,
 perc_vendas,
 top_80,
 top_90
from TEMP_FATO_OV a
inner join TEMP_TOP_MAT_FPP b ON a.COD_MATERIAL = b.COD_MATERIAL
Where {0}
""".format(filtro_fpp))
DATASET_FPP_PBR1_PS.registerTempTable("DATASET_FPP_PBR1_PS")

In [0]:
try: #entra aqui caso o arquivo ja exista
  dbutils.fs.ls(path_ml+'DATASET_FPP_PBR1_PS')
  
  #replace caso ja exista dado em alguma data
  DATASET_FPP_PBR1_PS.withColumn("VAL_ORDEM_BRUTO",DATASET_FPP_PBR1_PS.VAL_ORDEM_BRUTO.cast('double'))\
    .withColumn("perc_vendas",DATASET_FPP_PBR1_PS.perc_vendas.cast('double'))\
    .write \
    .format("delta") \
    .option("delta.autoOptimize.optimizeWrite","true")\
    .option("delta.autoOptimize.autoCompact" , "true")\
    .format("delta") \
    .mode("overwrite") \
    .save(path_ml+'DATASET_FPP_PBR1_PS')

except Exception as e: #entra aqui caso seja a primeira execucao - arquivo nao existe e ira ser criado
  if 'java.io.FileNotFoundException' in str(e):
    DATASET_FPP_PBR1_PS.withColumn("VAL_ORDEM_BRUTO",DATASET_FPP_PBR1_PS.VAL_ORDEM_BRUTO.cast('double'))\
    .withColumn("perc_vendas",DATASET_FPP_PBR1_PS.perc_vendas.cast('double'))\
    .write \
    .format("delta") \
    .option("delta.autoOptimize.optimizeWrite","true")\
    .option("delta.autoOptimize.autoCompact" , "true")\
    .save(path_ml+'DATASET_FPP_PBR1_PS')

In [0]:
ds_clientes_recomendar = spark.sql("""
select 
  distinct CLI.COD_CLIENTE as cli
        
from FATO_OV OV
inner join DIM_CLIENTE CLI      ON OV.DIM_CLIENTE_ID = CLI.DIM_CLIENTE_ID
AND to_date(OV.DAT_ORDEM,'yyyyMMdd') BETWEEN CLI.DATA_INICIO_VIGENCIA AND COALESCE(CLI.DATA_FIM_VIGENCIA,'9999-12-31')
inner join DIM_MATERIAL MAT     ON OV.DIM_MATERIAL_ID = MAT.DIM_MATERIAL_ID
inner join ds_recomendacoes REC ON Organizacao = OV.COD_ORGANIZACAO_VENDA
AND lpad(trim(cod_cli),10,'0') = CLI.COD_CLIENTE
AND lpad(trim(cod_mat),18,'0') = MAT.COD_MATERIAL
Where OV.DAT_ORDEM >= "{0}"
AND REC.REGISTRO_ATUAL = 1
""".format(Current_Date))

In [0]:
try: #entra aqui caso o arquivo ja exista
  dbutils.fs.ls(path_ml+'ds_clientes_recomendar')
  
  #replace caso ja exista dado em alguma data
  ds_clientes_recomendar.write \
    .format("delta") \
    .mode("overwrite") \
    .option("delta.autoOptimize.optimizeWrite","true")\
    .option("delta.autoOptimize.autoCompact" , "true")\
    .save(path_ml+'ds_clientes_recomendar')

except Exception as e: #entra aqui caso seja a primeira execucao - arquivo nao existe e ira ser criado
  if 'java.io.FileNotFoundException' in str(e):
    ds_clientes_recomendar.write \
    .format("delta") \
    .option("delta.autoOptimize.optimizeWrite","true")\
    .option("delta.autoOptimize.autoCompact" , "true")\
    .save(path_ml+'ds_clientes_recomendar')

In [0]:
spark.sql("""
select 
  distinct count( distinct CLI.COD_CLIENTE) as cli
        
from FATO_OV OV
inner join DIM_CLIENTE CLI      ON OV.DIM_CLIENTE_ID = CLI.DIM_CLIENTE_ID
AND to_date(OV.DAT_ORDEM,'yyyyMMdd') BETWEEN CLI.DATA_INICIO_VIGENCIA AND COALESCE(CLI.DATA_FIM_VIGENCIA,'9999-12-31')
inner join DIM_MATERIAL MAT     ON OV.DIM_MATERIAL_ID = MAT.DIM_MATERIAL_ID
inner join ds_recomendacoes REC ON Organizacao = OV.COD_ORGANIZACAO_VENDA
AND lpad(trim(cod_cli),10,'0') = CLI.COD_CLIENTE
AND lpad(trim(cod_mat),18,'0') = MAT.COD_MATERIAL
Where OV.DAT_ORDEM >= "{0}"
AND REC.REGISTRO_ATUAL = 1
""".format(Current_Date)).show()

In [0]:
%run /ML/PEDIDO_SUGERIDO/01-Recomendação_Varejo_Graphlab_Mat_6M

In [0]:
%run /ML/PEDIDO_SUGERIDO/02-Recomendação_FPP2_Graphlab_Mat_06_Meses

In [0]:
%run /ML/PEDIDO_SUGERIDO/00-Pedido_Sugerido_Origens_Dados_12_Meses

In [0]:
%run /ML/PEDIDO_SUGERIDO/01-Recomendação_Varejo_Graphlab_Mat_12M

In [0]:
%run /ML/PEDIDO_SUGERIDO/02-Recomendação_FPP2_Graphlab_Mat_12_Meses